In [1]:
import json
import requests
import difflib
import pandas as pd

In [3]:
api_key = ""

def getData(url):
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text.encode('utf-8'))
    print(json.loads(response.text.encode('utf-8')))

In [4]:
name = "TGG EXTRAS"
url = f"https://www.googleapis.com/youtube/v3/search?key={api_key}&q={name}&part=snippet"
data = getData(url)
print("user/channel name :", name)

{'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
user/channel name : TGG EXTRAS


In [ ]:
channels = set()
for channel in data['items']:
    channels.add(
        json.dumps(
            {
                "channelId" :channel.get('snippet',dict()).get('channelId',''),
                "channelTitle" :channel.get('snippet',dict()).get('channelTitle','')
            }
        )
    )
channels = [json.loads(channel) for channel in channels]
for channel in channels:
    if difflib.SequenceMatcher(None, name.lower(), channel.get('channelTitle','').lower()).ratio()*100 >= 90:
       channelId = channel.get('channelId','')
print("channelId :", channelId)

TypeError: ignored

In [ ]:
curl = f"https://www.googleapis.com/youtube/v3/channels?part=statistics&part=snippet&id={channelId}&key={api_key}"
cdata = getData(curl)

In [ ]:
channel = cdata['items'][0]
ChannelInfo = {
    "channelId" : channel.get('id',''),
    "channelTitle" : channel.get('snippet',dict()).get('title',''),
    "channelDescription" : channel.get('snippet',dict()).get('description',''),
    "publishedAt" : channel.get('snippet',dict()).get('publishedAt',''),
    "subscriberCount" : channel.get('statistics',dict()).get('subscriberCount',''),
    "videoCount" : channel.get('statistics',dict()).get('videoCount',''),
    "viewCount" : channel.get('statistics',dict()).get('viewCount',''),
}
print("ChannelInfov :", ChannelInfo)

{'channelDescription': "(FAMILY FRIENDLY YOUTUBER😁)\n\nTHIS CHANNEL DOESN'T HAVE ANY  CONTENT \n\n______________________________\n\n\nEmail for Bussiness Enquiries  ► workwithTgg@gmail.com 😄",
 'channelId': 'UCwfrESIkoS37-0UPE3HILTg',
 'channelTitle': 'TGG EXTRAS',
 'publishedAt': '2018-02-13T17:43:03Z',
 'subscriberCount': '82600',
 'videoCount': '44',
 'viewCount': '5886072'}

In [ ]:
cvdata = []
nextPageToken = ""
while True: 
    csurl = f"https://www.googleapis.com/youtube/v3/search?channelId={channelId}{nextPageToken}&part=id&maxResults=30&key={api_key}"
    csdata = getData(csurl)
    cvdata.append(csdata)
    nextPageToken = f"&pageToken={csdata.get('nextPageToken', '')}"
    if not nextPageToken:
        break
print("cvdata :", len(cvdata))

{'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}


AttributeError: ignored

In [ ]:
videoIds=[]
playlistIds=[]
for v in cvdata:
    video = v['items']
    if video['id'].get('videoId',''):
        videoIds.append(video['id'].get('videoId',''))
    elif video['id'].get('playlistId',''):
        playlistIds.append(video['id'].get('playlistId',''))
print("playlistIds :",len(playlistIds))
print("videoIds :", len(videoIds))

['PLN99XDk2SYr5zUoK1Ie8hBwT8nolAND8_',
 'PLN99XDk2SYr5ybr4HjH7EEyJRlwCwQtaj',
 'PLN99XDk2SYr7ByF11pSu8Hqxbx066h44H',
 'PLN99XDk2SYr4Vux2b7g04_yKXxpYq6JXp',
 'PLN99XDk2SYr4Yx4lJCSCCiMdwiD-bO7z1',
 'PLN99XDk2SYr5CRS7c1hk62yEultl8KaYn',
 'PLN99XDk2SYr7wPIfqNrgwbWNBvwYvohWU',
 'PLN99XDk2SYr7ff38RClKJQy0478Z3RqdP',
 'PLN99XDk2SYr4qLc5beFxiMwpdtIptW5li',
 'PLN99XDk2SYr60cKz_JpegpftOTRzfFNFt',
 'PLN99XDk2SYr4NATZeMxshFcg8JRn1-h1S',
 'PLN99XDk2SYr4EqI6foftUGsztysm1HfV1',
 'PLN99XDk2SYr5fOd87hQryXZ_DP5ndK-8Y']

In [ ]:
videoData = []
for id in videoIds:
    vurl = f"https://www.googleapis.com/youtube/v3/videos?part=id&part=snippet&part=statistics&part=contentDetails&id={id}&key={api_key}"
    vdata = getData(vurl)
    videoData.append(vdata)

In [ ]:
output = []
for v in videoData:
    video = v['items']
        output.append(
            {   
                "channelId" : video.get('snippet',dict()).get('channelId',''),
                "channelTitle" : video.get('snippet',dict()).get('channelTitle',''),
                "videoId" : video.get('id',''),
                "videoUrl" : f"https://www.youtube.com/watch?v={video.get('id','')}",
                "title" : video.get('snippet',dict()).get('localized',dict()).get('title',''),
                "description" : video.get('snippet',dict()).get('description',''),
                "publishedAt" : video.get('snippet',dict()).get('publishedAt',''),
                "duration" : video.get('contentDetails',dict()).get('duration',''),
                "viewCount" : video.get('statistics',dict()).get('viewCount',''),
                "likeCount" : video.get('statistics',dict()).get('likeCount',''),
                "dislikeCount" : video.get('statistics',dict()).get('dislikeCount',''),
                "commentCount" : video.get('statistics',dict()).get('commentCount',''),
                "tags" : "",
                "embedvideo" : f"""<iframe width="1440" height="762" src="https://www.youtube-nocookie.com/embed/{video.get('id','')}" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>"""
            }
        )

In [ ]:
df = pd.DataFrame(output)
df.to_csv("videos.csv", index=False, header=output[0].keys())